In [0]:
# Verifique se o esquema existe, caso contrário, crie-o
spark.sql("CREATE SCHEMA IF NOT EXISTS api_cerveja.2_silver")

spark.sql("DROP TABLE IF EXISTS api_cerveja.2_silver.tb_cerveja_limpo")

DataFrame[]

In [0]:
spark.sql('''
    CREATE TABLE api_cerveja.2_silver.tb_cerveja_limpo (
    id_brewery STRING COMMENT 'Identificador único da Cervejaria',
    name_brewery STRING COMMENT 'Nome da Cervejaria',
    brewery_type STRING COMMENT 'Tipo de Cervejaria',
    address_1 STRING COMMENT 'Endereço da Cervejaria',
    address_2 STRING COMMENT 'Endereço da Cervejaria',
    city STRING COMMENT 'Cidade da Cervejaria',
    state_province STRING COMMENT 'Estado da Cervejaria',
    postal_code STRING COMMENT 'CEP da Cervejaria',
    country STRING COMMENT 'País da Cervejaria',
    longitude DOUBLE COMMENT 'Longitude da Cervejaria',
    latitude DOUBLE COMMENT 'Latitude da Cervejaria',
    phone STRING COMMENT 'Telefone da Cervejaria',
    website_url STRING COMMENT 'Site da Cervejaria',
    state STRING COMMENT 'Estado da Cervejaria',
    street STRING COMMENT 'Rua da Cervejaria'
)
COMMENT 'Esta tabela é uma entidade corporativa que contém cervejarias do EUA'
''')

DataFrame[]

In [0]:
%sql
INSERT INTO api_cerveja.2_silver.tb_cerveja_limpo
SELECT
    id AS id_brewery,
    name AS name_brewery,
    brewery_type AS brewery_type,
    address_1 AS address_1,
    address_2 AS address_2,
    city AS city,
    state_province AS state_province,
    postal_code AS postal_code,
    country AS country,
    longitude AS longitude,
    latitude AS latitude,
    phone AS phone,
    website_url AS website_url,
    state AS state,
    street AS street
FROM api_cerveja.1_bronze.tb_cerveja;


num_affected_rows,num_inserted_rows
50,50


In [0]:
spark.sql('''
ALTER TABLE api_cerveja.2_silver.tb_cerveja_limpo ADD COLUMNS (
  data_carga DATE,
  data_hora_carga TIMESTAMP
);
''')

DataFrame[]

In [0]:
%sql
UPDATE api_cerveja.2_silver.tb_cerveja_limpo
SET 
  data_carga = current_date(),
  data_hora_carga = from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo');

num_affected_rows
50


In [0]:
%sql
SELECT COUNT(*) FROM api_cerveja.2_silver.tb_cerveja_limpo WHERE address_2 IS NULL;

COUNT(*)
49


In [0]:
spark.sql('''
ALTER TABLE api_cerveja.2_silver.tb_cerveja_limpo
SET TBLPROPERTIES (
  'delta.minReaderVersion' = '2',
  'delta.minWriterVersion' = '5',
  'delta.columnMapping.mode' = 'name'
);
''')

DataFrame[]

In [0]:
%sql
-- COMO SO TEMOS 1 LINHA COM VALOR NA COLUNA ADDRESS_2 ELA SE TORNA INSIGNIFICANTE PARA MIM, LOGO VAMOS REMOVE-LA
ALTER TABLE api_cerveja.2_silver.tb_cerveja_limpo 
DROP COLUMN address_2;

In [0]:
%sql
-- DUPLICADAS
ALTER TABLE api_cerveja.2_silver.tb_cerveja_limpo 
DROP COLUMN state;

In [0]:
%sql
-- DUPLICADAS
ALTER TABLE api_cerveja.2_silver.tb_cerveja_limpo 
DROP COLUMN street;

In [0]:
%sql
OPTIMIZE api_cerveja.2_silver.tb_cerveja_limpo;

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1754595715066, 1754595717588, 8, 0, null, List(0, 0), null, 14, 14, 0, 0, null)"


In [0]:
%sql
VACUUM api_cerveja.2_silver.tb_cerveja_limpo 
RETAIN 168 HOURS;

path
""
